<a href="https://colab.research.google.com/github/EnisBerk/speech_audio_understanding/blob/master/notebooks/audioset_inference_basic_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pathlib import Path



In [2]:
my_file = Path("./md_50000_iters.tar")
if not my_file.is_file():
  !curl https://storage.googleapis.com/deep_learning_enis/speech_audio_understanding/Eusipco2018_Google_AudioSet/md_50000_iters.tar --output ./md_50000_iters.tar
  !curl https://gist.githubusercontent.com/EnisBerk/19d817e9a6a9c060465a5b95c8d54d97/raw/8e0c24e92853e779e222bc3a152d7d65351cfe52/model.py --output ./model.py
  !curl https://raw.githubusercontent.com/qiuqiangkong/audioset_classification/master/metadata/class_labels_indices.csv --output ./class_labels_indices.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.8M  100 55.8M    0     0  19.9M      0  0:00:02  0:00:02 --:--:-- 19.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5748  100  5748    0     0  15166      0 --:--:-- --:--:-- --:--:-- 15126
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14675  100 14675    0     0  36779      0 --:--:-- --:--:-- --:--:-- 36779


In [0]:
from model import *

freq_bins = 128
classes_num = 527

# Hyper parameters
hidden_units = 1024
drop_rate = 0.5

model = FeatureLevelSingleAttention(
            freq_bins, classes_num, hidden_units, drop_rate)

In [4]:
checkpoint = torch.load("md_50000_iters.tar")
model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
iteration = checkpoint['iteration']

model.eval()

FeatureLevelSingleAttention(
  (emb): EmbeddingLayers(
    (conv1): Conv2d(128, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv3): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (attention): Attention(
    (att): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
    (cla): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
  )
  (fc_final): Linear(in_features=1024, out_features=527, bias=True)
  (bn_attention): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [6]:
# here is how to make predictions with model:
y_pred=model(input_sound)
# however input should be in shape of (numer of samples,length of each sound sample,filter count) 
#filter count is 128 
# I wrote following function to work on our audisoet easly.
# Reshapes input accordingly and returns k highest classes for  each sample


NameError: ignored

In [0]:
#input_tensor should be in shape of (sample_count,10)


# use it as following:
#y_preds,y_preds_index = inference(input_tensor,batch_size=500,first_k=10)
# explanation:
# y_preds are the values assigned to each class predicted 
# y_preds_index are the indexes of those classes 
# first_k controls how many classes will be returned, only highest scoring classes are returned


def inference(input_x,batch_size=500,first_k=1):
  predicted_labels=[]
  y_preds=torch.empty(0,first_k)
  y_pred_indexs=torch.empty((0,first_k),dtype=torch.long)
  start=0
  end=input_x.shape[0]-batch_size
  left=input_x.shape[0]%batch_size
  if batch_size%10!=0:
    print("ERROR batch_size should be divisible by 10")
    return None,None
  for i in range(0,end,batch_size):
    y_pred=model(input_x[i:i+batch_size].reshape(int(batch_size/10),10,128))
    y_pred, y_pred_index = torch.topk(y_pred, first_k, dim=1, largest=True, sorted=True)
    y_preds=torch.cat((y_preds,y_pred),0)
    y_pred_indexs=torch.cat((y_pred_indexs,y_pred_index),0)
    
  if left>0:
    y_pred=model(input_x[-left:].reshape(int(left/10),10,128))
    y_pred, y_pred_index = torch.topk(y_pred, first_k, dim=1, largest=True, sorted=True)
    y_preds=torch.cat((y_preds,y_pred),0)
    y_pred_indexs=torch.cat((y_pred_indexs,y_pred_index),0)
    
  return y_preds,y_pred_indexs
